In [ ]:
# Order preparation for NSE

# STATUS: Completed
# Run-time: 10 seconds

# Dependencies:
# /zdata/*.pkl - for pickles generated by 01_nse_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=11)

In [ ]:
import datetime
import pandas as pd
import numpy as np
from os import listdir
import math

#...assignments
m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
base = 0.05       # Upper or Lower base multiple for prices

min_rom = 0.8
min_pop = 0.9

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

#...get current positions
#________________________

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#...harvest preparation
#______________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
#     if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = (util.parseIBDatetime(expiry) - datetime.datetime.now().date()).days
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
df_p['dte'] = [(util.parseIBDatetime(d.lastTradeDateOrContractMonth) - datetime.datetime.now().date()).days for d in df_p.contract]

discount = [m for m in map(expPricePct, expiry)]
df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
df_p['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_p.position, df_p.hvstPrice)]

# ignore data for dte < 3 days. These are as good as gone.
df_h = df_p[df_p.dte > 3].reset_index(drop=True)

hqc = ib.qualifyContracts(*df_h.contract)
df_h = df_h.assign(qual_contract=hqc)

# ...sowing prepration
#_____________________

#... get the lots and margins
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])

#... make the blacklist

df1 = df_slm.groupby('ibSymbol').sum()
df1['used_margin'] = -df1.position * df1.TotMgnPerShr
df1['max_margin'] = max_p
df1['avail_margin'] = df1.position * df1.TotMgnPerShr + df1.max_margin

df1['max_units'] = (df1.avail_margin/(df1.Mlot*df1.TotMgnPerShr)).apply(np.floor)

blacklist = list(df1[df1.max_units <= 0].index)

#...build the high-pop-roc dataframe
fs = listdir('./zdata/')

opts = ([f[:-8]+'_opt.pkl' for f in fs if f[-8:] == '_opt.pkl'])
ohlcs = ([f[:-8]+'_ohlc.pkl' for f in fs if f[-8:] == '_opt.pkl'])
unds = ([f[:-8]+'_und.pkl' for f in fs if f[-8:] == '_opt.pkl'])

In [40]:
df_opt = pd.concat([pd.read_pickle('./zdata/'+f) for f in opts], axis=0, sort=True).reset_index(drop=True).sort_values('rom', ascending=False)
df_ohlc = pd.concat([pd.read_pickle('./zdata/'+f).reset_index() for f in ohlcs], axis=0, sort=True)
df_und = pd.concat([pd.read_pickle('./zdata/'+f) for f in unds])

# remove options in black list
df_opt = df_opt[~df_opt.symbol.isin(blacklist)]

# get the lots
df_mlot = df_und[['symbol', 'lot']]
df_opt1 = df_opt.merge(df_mlot, on='symbol', how='inner')

# arrange the columns
cols = ['symbol', 'right', 'expiry', 'dte', 'strike', 'undPrice', 'lo52',  'hi52', 
'stdev', 'volatility', 'margin', 'lot', 'bsmPrice', 'pop', 'rom', 'price', 'option']

df_opt1 = df_opt1[cols]

# take only high rom and pops
df_opt1 = df_opt[(df_opt.rom > min_rom) & 
                   (df_opt.dte > 2) &
                   (df_opt['pop'] > min_pop)]

df_opt2['expPrice'] = round(round((df_opt2.price+0.1)/ base) * base, -int(math.floor(math.log10(base))))

# make the price equal to bsm price for those whose expPrice is low
df_opt2.loc[df_opt2.expPrice < df_opt2.bsmPrice, 'expPrice'] = df_opt2.bsmPrice

# keep only those options that are beyond 52 week high and low
hilo_mask = ((df_opt2.right == 'P') & (df_opt2.strike < df_opt2.lo52)) | ((df_opt2.right == 'C') & (df_opt2.strike > df_opt2.hi52))
df_opt3 = df_opt2[hilo_mask]

# Make df the dataframe that you want to execute on!
df = df_opt3.copy()   # make this the last dataframe to get the orders placed

df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2

contracts = [c for c in df.option]

ass_limit = max(df.strike*df.lot)+100  # assumes how much it will cost if the strike gets assigned (theoretical for NSE)

df = df.assign(qty=pd.concat([round(ass_limit / (df.strike * df.lot)), 
                         round(max_p/df.margin), 
                         pd.Series(5, index=df.index)], axis=1).min(axis=1))
    
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(contracts), \
      len(df.symbol.unique()), sum(df.margin*df.qty), av_funds*max_nlvp, sum(df.expPrice*df.lot*df.qty)))

40 contracts from 28 scrips, consuming 7,763,272 margin from full available funds of 3,704,976, giving INR 942,826


In [41]:
# ...review calls and puts
# e.g. for puts from path: C:\Users\kashir\Documents\IBKR\nse\zdata\putswatch.csv

cols = ['right', 'symbol', 'strike', 'undPrice', 'dte', 'pop', 'rom', 'price', 'expPrice', 'margin', 'lot', 'qty', 'option']
# df[cols].sort_values(['right', 'symbol', 'strike'])
gb = df[cols].groupby('right')
df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
df = pd.concat([df_puts, df_calls]).reset_index(drop=True)

# get the output
df.to_csv('./zdata/check.csv', index=None, header=True)

# Write to watchlist. This watchlist is to be checked in tws / tradingview for the lowest strike and expiry
# This needs to be imported to IBKR's watchlist
watchputs = [('DES', s, 'STK', 'NSE') for s in df_puts.symbol.unique()]
df_wp = util.df(watchputs)

df_wp.to_csv('./zdata/putswatch.csv', index=None, header=False)

watchcalls = [('DES', s, 'STK', 'NSE') for s in df_calls.symbol.unique()]
df_wp = util.df(watchcalls)

df_wp.to_csv('./zdata/callswatch.csv', index=None, header=False)

In [42]:
# After going through checked.csv, with puts and calls, eliminate risky options
# Save the file as checked.csv

df_final = pd.read_csv('./zdata/checked.csv') # picks up the checked and ready-to-go contracts
cs = [eval(c) for c in df_final.option]  # convert the "quoted strings" from csv back to object
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df_final.lot, df_final.expPrice)]
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(cs), \
      len(df_final.symbol.unique()), sum(df_final.margin), av_funds*max_nlvp, sum(df_final.expPrice*df_final.lot)))

40 contracts from 28 scrips, consuming 3,411,360 margin from full available funds of 3,704,976, giving INR 440,748
